---

Previously, I've discussed bounding boxes and how they work to improve efficiency when evaluating small models over a large range. Since then, I've added a `bounding_box` property to the `astropy.modeling.Model` class that defines the range over which a model is significant. All built-in models that have a finite spatial extent will eventually be bounded by default. The Gaussian models are also bounded at 5.5-sigma corresponding to an integrated error < 1e-7. For the most part, these limits are easy to derive from a model's characteristic width, but it turns out that the rotated 2D ellipse (or 2D Gaussian) is not so straightforward. After spending the better part of an hour drawing boxes and silently chanting "soh-cah-toa", I had a sufficiently close approximation that only erred on the large side for some rotation angles. Axel later directed me to the collective programming brain-dump otherwise known as [stackoverflow](http://stackoverflow.com/questions/87734/how-do-you-calculate-the-axis-aligned-bounding-box-of-an-ellipse), where the complexity of the correct solution made me feel slightly better about my trigonometric failure. 

This property is a useful attribute to have available for a variety of purposes, but it was developed as a necessary prerequiste for fast model rasterization. The wary user can be rest assured that the `bounding_box` will not limit the model's evaluation in any way unless it is evaluated with the `render_model` function (a topic for another post). As such, with array indexing in mind, the ordering of the limits was chosen to be consistent with the `numpy` indexing convention. This is, confusingly, the reverse of the model input. This seems to be a generally accepted deviation from the Zen of Python which states, "There should be one-- and preferably only one --obvious way to do it". The reasoning for it is [well explained](http://docs.scipy.org/doc/numpy/reference/internals.html#multidimensional-array-indexing-order-issues), and comes down to a fundamental conflict between how humans think and how computers think, so what's "obvious" depends on who you ask. 

The user can manuall set the `bounding_box` limits for any model, like `model.bounding_box = ((y_low, y_high), (x_low, x_high))`. If the desired limits are +/- infinity, the `bounding_box` should be turned off by setting `model.bounding_box = None`. Lastly, the default limits can always be (re)set with  `model.bounding_box = 'auto'` or accessed directly through model-specific `bounding_box_default` method. This method in turn can be explicitly (re)defined, which is especially useful for `custom_model` instances. 

Though simply in theory, the `bounding_box` implementation took quite a bit of time, thought, and discussion to get right. While there is still some room for improvement; the docs are written, the tests are passing and one major hurdle to fast model rasterization with Astropy has been cleared. 


---

The `render_model` function is what executes model rasterization, and the `bounding_box`'s are what make it fast. `Render_model` will evaluate any model on an image/array or coordinate grid. It's easiest to think about 1D and 2D models, but this functionality supports models in N-dimensionsal space. If a model's `bounding_box` is set, the evaluation is limited to the bounded region. This also functions to produce "postage stamps" of the model, if no other inputs are given. While `render_model` currently exists as a core function, this may eventually morph into a model method, i.e. `Model.render(image)` rather than `render_model(model, image)`. However that is for future work. Having the core functionality in a stable state with detailed documentation and several tests pasting, this PR is well into the final review stage, and should be merged any day. In the meantime, I'm already working on a follow up PR that, among other things, adds WCS support. 

In the example below, I 

In [ ]:
## A 2D source with a 3rd frequency dimension
%pylab inline
import os
os.chdir('/Users/Patti/gsoc/astropy')

from astropy.wcs import WCS
from astropy.modeling import models, render_model

# Define a 3D custom model that represents a 2D source with a frequency dimension



w= WCS(naxis=3)
w.wcs.crval = [180., 0., 550.]
w.wcs.crpix = [150, 100, 50]
w.wcs.cdelt = [.25, .25, 10]
w.wcs.ctype = ["RA---SIN", "DEC--SIN", "FREQ"]

rlim,dlim,flim = w.wcs_pix2world([0,imshape[2]],[0,imshape[1]],
                            [0,50],1)
                                 
def source3d(ra, dec, freq, amp_0=1, ra_0=180., dec_0=0., 
             freq_0=400., width=4., alpha=-1.):
    amp = amp_0 * (freq/freq_0)**alpha
    g = models.Gaussian2D(amp, ra_0, dec_0, width, width, 0)
    return g(ra,dec)

def source3d_bbox(self):
    g = models.Gaussian2D(self.amp_0, self.ra_0, self.dec_0, 
                          self.width, self.width, 0)
    dlim, rlim = g.bounding_box
    return ((self.freq_0.value, self.freq_0*2), dlim, rlim)

Source3D = models.custom_model(source3d)
Source3D.bounding_box_default = source3d_bbox

model = Source3D()
model.bounding_box = 'auto'
f1,f2,d1,d2,r1,r2 = np.array(model.bounding_box).flatten()

# Render the model image
imshape = (101, 201, 301)
im =render_model(model,arr=np.zeros(imshape),wcs=w)

plt.figure(figsize=(12,16))
plt.subplots_adjust(left=.05,right=.95,hspace=.05,top=.95,bottom=.05)
plt.subplot(211)
plt.imshow(im[34,:,:],origin='lower',vmax=1,vmin=0,
           extent=[rlim[0],rlim[1],dlim[0],dlim[1]])
plt.plot([r1,r2,r2,r1,r1],[d1,d1,d2,d2,d1],'w-',lw=5,alpha=.25)
plt.axis([rlim[0],rlim[1],dlim[0],dlim[1]])
plt.xlabel('RA')
plt.ylabel('Dec')

plt.subplot(212)
plt.plot(np.linspace(flim[0],flim[1],101), im[:,100,150], lw=5)
plt.xlim(flim[0],flim[1])
plt.xlabel('Freq')
plt.ylabel('Amp')

---

While `render_model` is a great way to quickly "image" models, the models are assumed to be defined in pixel coordinates. By adding WCS support, `render_model` can accept a model that is defined in world coordinates (e.g. (RA, Dec), or (longitude, latitude)) so long as an `astropy.wcs.WCS` object is passed that defines the transformation. This works very well, and it simplifies the astro-user experience. An astronomer can go directly from a source catalog -with (RA, Dec, Flux) and some angular parameters depending on the model - to a model image. While defining the WCS object may be the tricky part, this is readily available from the header of most astronomical data images stored as fits files. Examples are also available online - something I found useful when writing examples and tests. 

Again, this functionality will work for N-dimensional models so long as the projections are supported by astropy. Common use cases include a 1D energy spectrum, or a 3D image cube with a redshift or frequency axis. At this point I'm just finishing up the tests and documentation. Additional improvements in the new PR will include 1. defining the `bounding_box_default` functions for all existing models where it makes sense, 2. editing the newly minted `Cutout2D` class- which also has a bounding box attribute- for consistencey and seamless interaction, 3. Allowing bounds to be set for only some dimensions of a multi-dimensional model.

In [ ]:
allow user to set limits to infinity. Becomes None. Allow partially set limits e.g. ((0,100),None) or (None,100).